### First Follow

In [13]:
import os
import pandas as pd
import numpy as np

In [14]:
file_path = "lib/input/"
file_name = "input_4.txt"
try:
    codigo_entrada = open(file_path + file_name, "r", encoding="utf8").read()
except:
    codigo_entrada = """S -> aSbc | D
D -> dD | d"""

In [15]:
print(codigo_entrada)

Z -> dY
Y -> XTY | λ
T -> c | λ
X -> T | λ


In [16]:
import re

In [17]:
qtd_linhas = codigo_entrada.count("\n") + 1
qtd_linhas

4

In [18]:
### Encontrar
regex_terminais = r"[a-z+:=*-]"
matches = re.finditer(regex_terminais, codigo_entrada.replace("->"," "), re.MULTILINE)
variaveis = []
for match in matches:
    variaveis.append(match.group())

variaveis_terminais = list(set(variaveis))

variaveis_terminais

['d', 'c']

In [19]:
regex_nao_terminais = r"[A-Z]"
matches = re.finditer(regex_nao_terminais, codigo_entrada.replace("->"," "), re.MULTILINE)
variaveis = []
for match in matches:
    variaveis.append(match.group())

variaveis_nao_terminais = list(set(variaveis))
variaveis_nao_terminais

['Z', 'X', 'T', 'Y']

In [20]:
print(codigo_entrada)

Z -> dY
Y -> XTY | λ
T -> c | λ
X -> T | λ


In [58]:
production = dict()

In [60]:
production

{'Z': 'dY', 'Y': 'XTY | λ', 'T': 'c | λ', 'X': 'T | λ'}

In [59]:
produz_vazio = []

for line in codigo_entrada.splitlines():
    left = line.split('->')[0].strip() 
    right = line.split('->')[1].strip()
    # print(f"FIRST({left}) = FIRST({right})")
    print(f"FIRST({left}) = ", end="")

    production[left] = right

    for idx, right_part  in enumerate(right.split("|")): 

        if(right_part.strip() == "λ"): ## Guardar quando o elemento produz vazio
            produz_vazio.append(left) 

        if idx > 0:
            print(f" + FIRST({right_part.strip()})", end="")
        else:
            print(f"FIRST({right_part.strip()})", end="")        

    print("")

produz_vazio = list(set(produz_vazio))

FIRST(Z) = FIRST(dY)
FIRST(Y) = FIRST(XTY) + FIRST(λ)
FIRST(T) = FIRST(c) + FIRST(λ)
FIRST(X) = FIRST(T) + FIRST(λ)


In [22]:
produz_vazio

['X', 'T', 'Y']

In [72]:
a = 'T | λ'

In [74]:
[elem in a.split("|")]

['T ', ' λ']

In [75]:
def separar_prod(producao):
    saida = ""
    if len(producao) == 1:
        return first_set(producao)
    else:
        for prod in producao.split("|"):
            saida = saida + first_set(prod.strip())
        return saida

In [82]:
def first_set(prod):
    print(prod)
    if prod == "λ":
        return "λ"
    elif prod in variaveis_terminais:
        return prod
    elif len(prod) > 1 and prod[0] in variaveis_terminais:
        return prod[0]

    elif prod[0] in variaveis_nao_terminais and "λ" not in separar_prod(production[prod[0]]):    
        print("Regra Não Vazio")  
        print(prod[0])  
        return first_set(prod[0])

    elif prod[0] in variaveis_nao_terminais and "λ" in separar_prod(production[prod[0]]): 
        print("Regra Vazio") 
        print(prod[0], prod[1])
        return first_set(prod[0]) + first_set(prod[1])
    else:
        return print("Nenhuma Regra?")   


In [83]:
separar_prod("XTY")

XTY
T
c
λ
c
λ
Regra Vazio


IndexError: string index out of range

In [68]:
for line in codigo_entrada.splitlines():
    left = line.split('->')[0].strip() 
    right = line.split('->')[1].strip()

    print(f"FIRST({left}) = ", end="")

    for idx, right_part  in enumerate(right.split("|")): 

        if idx > 0:
            print(f" + {first_set(right_part.strip())}", end="")
        else:
            print(f"{first_set(right_part.strip())}", end="")        

    print("")



FIRST(Z) = dY
d
FIRST(Y) = XTY


AttributeError: 'list' object has no attribute 'strip'